In [ ]:
from langgraph.graph import StateGraph,START,END
from pydantic import BaseModel, Field
from typing import TypedDict,Annotated

from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()
import operator
api_key = os.getenv("GEMINI_API_KEY")

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                               api_key=api_key)

In [2]:
class evaluationSchema(BaseModel):
    feedback: str = Field(description="detailed feedback on the essay")
    score: int = Field(description="The score of the essay out of 10,ge= 0 and le=10")

In [3]:
structure_model  = model.with_structured_output(evaluationSchema)

In [4]:
essay = """The Rise of Artificial Intelligence in India

Artificial Intelligence (AI) has emerged as one of the most transformative technologies of the 21st century. Across the globe, it is reshaping industries, governance, education, and the way humans interact with machines. India, being the world’s most populous nation and a rapidly growing economy, has witnessed a remarkable rise in AI adoption and innovation. The journey of AI in India reflects a blend of government initiatives, private sector investments, academic contributions, and entrepreneurial zeal.

Growth and Opportunities

India’s digital ecosystem provides fertile ground for AI growth. With over 800 million internet users, the world’s largest pool of data scientists and engineers, and a thriving startup culture, the nation is uniquely positioned to become a global AI hub. AI applications in India span across diverse sectors such as healthcare, agriculture, education, finance, manufacturing, and governance. For example, AI-based crop monitoring systems help farmers improve yield, while AI-driven diagnostic tools assist doctors in early detection of diseases.

Government Initiatives

Recognizing the transformative potential of AI, the Government of India has launched several programs. The National Strategy for Artificial Intelligence (NSAI) by NITI Aayog envisions AI as a tool for “Inclusive Growth.” Initiatives such as Digital India, Make in India, Startup India, and Skill India further promote AI-driven innovations. Additionally, AI research centers and Centers of Excellence are being set up to encourage academic-industry collaboration.

Private Sector and Startups

India’s IT giants such as TCS, Infosys, and Wipro are investing heavily in AI research and services. Startups like Haptik, Mad Street Den, and SigTuple are creating world-class AI solutions in conversational AI, computer vision, and healthcare diagnostics. Global tech companies such as Google, Microsoft, and Amazon have also established AI research labs in India, recognizing its potential as both a talent hub and a growing market.

Challenges

Despite the rapid progress, India faces several challenges in fully realizing the benefits of AI. These include lack of high-quality datasets, limited adoption in rural areas, inadequate infrastructure, and concerns about job displacement due to automation. Moreover, ethical considerations such as privacy, transparency, and bias in AI systems require careful attention.

The Road Ahead

The future of AI in India lies in striking a balance between innovation and inclusivity. Strengthening AI education and skilling programs, building robust digital infrastructure, and promoting ethical AI practices will be critical. With the right policies, investments, and collaborations, India can harness AI not only for economic growth but also for social good, bridging gaps in healthcare, education, and governance.

Conclusion

The rise of Artificial Intelligence in India represents both an opportunity and a responsibility. It has the power to transform lives, empower citizens, and position India as a leader in the global AI revolution. By addressing challenges and fostering innovation, India can ensure that AI becomes a force for inclusive growth and sustainable development in the years to come."""

In [5]:
# promt = f"""You are an expert in evaluating essays. Your task is to provide detailed feedback and a score for the following essay:{essay}"""

In [6]:
# res = structure_model.invoke(promt)

In [7]:
# definition of the state

class UpscState(TypedDict):

    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_score: Annotated[list[int],operator.add]
    average_score: float

In [8]:
def evaluate_language(state: UpscState) -> UpscState:
    promt = f"""Evalute the language quality of the following essay and provide feedback and a score for the language quality:{state['essay']}"""
    res = structure_model.invoke(promt)
    return {"language_feedback": res.feedback,
            "individual_score": [res.score]}


In [9]:

def evaluate_analysis(state: UpscState) -> UpscState:
    promt = f"""Evalute the analysis quality of the following essay and provide feedback and a score for the analysis quality:{state['essay']}"""
    res = structure_model.invoke(promt)
    return {"analysis_feedback": res.feedback,
            "individual_score": [res.score]}

In [10]:
def evaluate_thought(state: UpscState) -> UpscState:
    promt = f"""Evalute the thought process quality of the following essay and provide feedback and a score for the thought process quality:{state['essay']}"""
    res = structure_model.invoke(promt)
    return {"clarity_feedback": res.feedback,
            "individual_score": [res.score]} 



In [11]:
def final_evaluation(state: UpscState) -> UpscState:
    avg_score  = sum(state["individual_score"]) / len(state["individual_score"])
    prompt  = f" baseed on the following feedback generate the final feedback \n Language Feedback: {state['language_feedback']}, Analysis Feedback: {state['analysis_feedback']}, Clarity Feedback: {state['clarity_feedback']}"
    overall_feedback = model.invoke(prompt).content
    return {"overall_feedback": overall_feedback,
            "average_score": avg_score}

In [12]:
#define the graph
graph =  StateGraph(UpscState)

# add nodes to the graph
graph.add_node("evaluate_language",evaluate_language)
graph.add_node("evaluate_analysis",evaluate_analysis)
graph.add_node("evaluate_thought",evaluate_thought)
graph.add_node("final_evaluation",final_evaluation)



# add edges to the graph
graph.add_edge(START,"evaluate_language")
graph.add_edge(START,"evaluate_analysis")
graph.add_edge(START,"evaluate_thought")
graph.add_edge("evaluate_language","final_evaluation")
graph.add_edge("evaluate_analysis","final_evaluation")
graph.add_edge("evaluate_thought","final_evaluation")
graph.add_edge("final_evaluation",END)

In [13]:
workflow = graph.compile()

In [14]:
initial_state = {"essay": essay}

# run the graph
result = workflow.invoke(initial_state)

In [15]:
result

{'essay': 'The Rise of Artificial Intelligence in India\n\nArtificial Intelligence (AI) has emerged as one of the most transformative technologies of the 21st century. Across the globe, it is reshaping industries, governance, education, and the way humans interact with machines. India, being the world’s most populous nation and a rapidly growing economy, has witnessed a remarkable rise in AI adoption and innovation. The journey of AI in India reflects a blend of government initiatives, private sector investments, academic contributions, and entrepreneurial zeal.\n\nGrowth and Opportunities\n\nIndia’s digital ecosystem provides fertile ground for AI growth. With over 800 million internet users, the world’s largest pool of data scientists and engineers, and a thriving startup culture, the nation is uniquely positioned to become a global AI hub. AI applications in India span across diverse sectors such as healthcare, agriculture, education, finance, manufacturing, and governance. For exam

In [16]:
print(result["overall_feedback"])
print(result["average_score"])

This essay offers a well-structured and informative overview of the rise of AI in India, employing clear, concise, and academic language suitable for a professional or academic audience.  The essay effectively uses examples to support its points, and the organization is logical and easy to follow.  However, several areas require strengthening to elevate the analysis and impact.

**Strengths:**

* **Clear Structure and Organization:** The essay progresses logically from introduction to conclusion, covering key aspects of AI's rise in India.
* **Comprehensive Overview:** The essay provides a good overview of government initiatives, private sector involvement, and challenges.
* **Academic Tone and Style:** The language is appropriate for a professional or academic audience.


**Areas for Improvement:**

* **Enhance Engagement and Impact:** The introduction needs more compelling hooks to immediately capture the reader's attention. The conclusion should be more impactful, offering a stronge